In [71]:
import os
import pandas as pd
cwd = os.getcwd()
inputFile = 'Farm Constraints Data Import Template - V1.0.xlsx'
import math

### Import Excel Input Data

In [72]:
# ===== Farm Inputs =====
# read farm input parameters
input_df = pd.read_excel(inputFile,sheet_name="Input Data")

# Replace empty cells with zero.

# Check for empty cells
first_row = input_df.iloc[0]
blank_cols = first_row[first_row.isna()].index.tolist()

if blank_cols:
    print("⚠️ Warning: Blank cells detected in the *Input Data* tab (first row):\n")
    for col in blank_cols:
        print(f"  → Column '{col}' is blank")
    print('Blank cells set to zero')

# Fill empty cells with 0
input_df = input_df.fillna(0)




#===== Technical Specs =====
# Read technical specs from excel file
specs_df = pd.read_excel(inputFile, sheet_name='Technical Specs')
# Check for empty cells
first_row = specs_df.iloc[0]
blank_cols = first_row[first_row.isna()].index.tolist()

if blank_cols:
    print("⚠️ Warning: Blank cells detected in the *Technical Specs* tab (first row):\n")
    for col in blank_cols:
        print(f"  → Column '{col}' is blank")
    print('Blank cells set to zero')

# Fill empty cells with 0
specs_df = specs_df.fillna(0)



# ===== Energy Bill =====
# Read energy bill from excel file
bill_df = pd.read_excel(inputFile, sheet_name='Energy Bill')
# Check for empty cells
first_row = bill_df.iloc[0]
blank_cols = first_row[first_row.isna()].index.tolist()

if blank_cols:
    print("⚠️ Warning: Blank cells detected in the *Energy Bill* tab (first row):\n")
    for col in blank_cols:
        print(f"  → Column '{col}' is blank")
    print('Blank cells set to zero')
# Fill empty cells with 0
bill_df = bill_df.fillna(0)

/Users/angelajordan/eaas-agri-food-decarbonisation/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/angelajordan/eaas-agri-food-decarbonisation/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Update Data Frames

In [73]:
# change column names for ease of use in code 
input_df.rename(columns=
                {"Available North Facing Roof Area for Panels (m2)": 'north_area',
                 'Available South Facing Roof Area for Panels (m2)': 'south_area',
                 'Available East Facing Roof Area for Panels (m2)': 'east_area',
                 'Available West Facing Roof Area for Panels (m2)': 'west_area',
                 'Roof Tilt': 'roof_tilt',
                 'Maximum Number of Batteries': 'max_batteries',
                 'Emissions Reduction Priority': 'emissions_priority',
                 'Costs Savings Priority': 'cost_priority',
                 "Grid Connection (kW)" : 'grid_connection',
                 "Location (x,y)": 'location',
                 "Start Year of Energy Demand Data (YYYY)": 'start_year',
                 "State": 'State'
                 }, inplace=True)
# convert to dictionary
inputs = input_df.iloc[0].to_dict()


# change column names for ease of use in code 
specs_df = specs_df.rename(columns=
                {"Cost of Battery ($/kWh)": 'battery_cost',
                 "Cost of Solar Panels ($/kWp)": 'solar_cost',
                 "Maintenance Cost (% of asset CAPEX)": 'om_cost',
                 "Discount Rate (%)": 'discount_rate',
                 "Battery Capacity (kWh)": 'battery_capacity',
                 "Battery Power rating (kW)": 'battery_power',
                 "Battery round trip efficiency (%)": 'round_trip_efficiency',
                 "Battery depth of discharge (%)": 'battery_dod',
                 "Number of battery cycles until 80% capacity": 'cycles_80',
                 "Battery lifespan (years)": 'battery_lifespan',
                 "Solar Panel Type ": 'panel_type',
                 "Solar panel roof  rack mounting type ": 'mounting_type',
                 "Solar panel dimensions (m)": 'panel_dimension',
                 "Solar panel lifespan (years)": 'panel_lifespan'
                })
# convert to dictionary
specs = specs_df.iloc[0].to_dict()


# change column names for ease of use in code 
bill_df = bill_df.rename(columns=
                         {"Peak Hours (24 hour time)": 'peak_hours',
                          "Off peak Rate (c/kWh)": 'off_peak_$',
                          "Peak period rate (c/kWh)": 'peak_$',
                          "LRET Environmental Charge (c/kWh)": 'LRET',
                          "SRES Environmental Charge (c/kWh)": 'SRES',
                          "VEEC Environmental Charge (Victoria Only) (c/kWh)": 'VEEC',
                          "Network Charge Rate (c/kWh)": 'network_charge',
                          "GST (%)": 'gst',
                          "Other (c/kWh)": 'other_charges'                     
                         })
# convert to dictionary
bill = bill_df.iloc[0].to_dict()

### Find Region from coordinates

In [74]:
# Create Polygons for Australian state borders 
def point_in_polygon(lat, lon, polygon):
    """Return True if (lat, lon) is inside polygon (list of (lat, lon) points)."""
    x, y = lon, lat
    inside = False
    n = len(polygon)
    for i in range(n):
        x1, y1 = polygon[i][1], polygon[i][0]
        x2, y2 = polygon[(i + 1) % n][1], polygon[(i + 1) % n][0]
        if ((y1 > y) != (y2 > y)) and (x < (x2 - x1) * (y - y1) / (y2 - y1) + x1):
            inside = not inside
    return inside

# Simplified polygons for each NEM region (clockwise lat/lon points)
NEM_POLYGONS = {
    "VIC1": [
        (-38.5, 141.0), (-39.1, 144.0), (-39.1, 149.0), (-37.5, 149.0),
        (-34.0, 141.0), (-35.5, 141.0), (-38.5, 141.0)
    ],
    "NSW1": [
        (-37.5, 141.0), (-29.0, 141.0), (-28.0, 153.6), (-36.0, 150.0),
        (-37.5, 149.0), (-37.5, 141.0)
    ],
    "QLD1": [
        (-29.0, 137.9), (-29.0, 153.6), (-10.0, 153.6), (-10.0, 137.9), (-29.0, 137.9)
    ],
    "SA1": [
        (-38.1, 129.0), (-25.9, 129.0), (-25.9, 141.0), (-38.1, 141.0), (-38.1, 129.0)
    ],
    "TAS1": [
        (-43.7, 144.0), (-43.6, 148.0), (-40.6, 148.0), (-40.6, 144.0), (-43.7, 144.0)
    ],
}

# Function to determine region of actual farm coordinates
def get_nem_region(lat, lon):
    """
    Returns the NEM region code (VIC1, NSW1, QLD1, SA1, TAS1)
    for a given latitude and longitude in Australia.

    If in NT/WA, defaults to NSW1 and prints a warning.
    """

    # Check if the point is inside one of the NEM state polygons
    for region, polygon in NEM_POLYGONS.items():
        if point_in_polygon(lat, lon, polygon):
            return region

    # Check WA
    if -35 <= lat <= -13 and 112 <= lon <= 129:
        print("⚠️ Warning: Coordinates are in Western Australia (WEM). Defaulting to NSW1.")
        return "NSW1"

    # Check NT
    if -26 <= lat <= -10 and 129 <= lon <= 138:
        print("⚠️ Warning: Coordinates are in Northern Territory (outside NEM). Defaulting to NSW1.")
        return "NSW1"

    print("⚠️ Warning: Coordinates appear outside recognised NEM or Australian regions. Defaulting to NSW1.")
    return "NSW1"



### Energy Bill Calculations

In [75]:
GST = bill['gst'] / 100

additional_energy_charges = (
    bill['LRET'] +
    bill['SRES'] +
    bill['VEEC'] +
    bill['network_charge'] +
    bill['other_charges']
) + GST * (
    bill['LRET'] +
    bill['SRES'] +
    bill['VEEC'] +
    bill['network_charge'] +
    bill['other_charges']
)

peak_energy_price = bill['peak_$'] + bill['peak_$']*GST
off_peak_energy_price = bill['off_peak_$'] + bill['off_peak_$']*GST

# Create List of charges for each month (c/kWh)
elec_bill_charges= [
    additional_energy_charges,  # jan
    additional_energy_charges,  # feb
    additional_energy_charges,  # mar
    additional_energy_charges,  # apr
    additional_energy_charges,  # may
    additional_energy_charges,  # jun
    additional_energy_charges,  # jul
    additional_energy_charges,  # aug
    additional_energy_charges,  # sep
    additional_energy_charges,  # oct
    additional_energy_charges,  # nov
    additional_energy_charges   # dec
]

peak = bill['peak_hours']
peak_hours = [tuple(map(float, peak.split(',')))]

### Solar Variables

Data on common panel types available in Australia

In [76]:
dim = specs['panel_dimension']                                      # Extracting dimensions from dataframe
panel_dimensions = tuple(map(float, dim.split(',')))                # set panel dimensions as a tuple

# Panel types dictionary with more detailed specifications
panel_types = {
    "monocrystalline": {
        "efficiency": 0.22, 
        "dimensions": panel_dimensions, 
        "temp_coefficient": -0.0035, 
        "watts_per_panel": 370,
        "v_mp": 33.7,  # Vmpp (V)
        "i_mp": 10.97,  # Impp (A)
        "technology": "monosi"  # For PVlib module database
    },
    "polycrystalline": {
        "efficiency": 0.18, 
        "dimensions": panel_dimensions, 
        "temp_coefficient": -0.0040, 
        "watts_per_panel": 330,
        "v_mp": 31.2,
        "i_mp": 10.58,
        "technology": "multisi"
    },
}

Data on common mounting configurations for solar panels

In [77]:
# Mounting configurations
mounting_configurations = {
    "roof_flush": {
        "mounting_type": "roof",
        "racking_model": "flush",
        "air_gap": 0.05,  # 5 cm air gap between panel and roof surface
        "temp_model_params": {"a": -3.56, "b": -0.075, "deltaT": 3},
        "ground_coverage_ratio": 0.98
    },
    "roof_tilt": {
        "mounting_type": "roof",
        "racking_model": "open_rack",
        "air_gap": 0.15,  # 15 cm air gap
        "temp_model_params": {"a": -3.47, "b": -0.0594, "deltaT": 3},
        "ground_coverage_ratio": 0.95
    }
}

### Site Parameters

Roof space available on site

In [ ]:
input_tilt = inputs['roof_tilt'] # Subject to change from input file
tilt_options = {'Flat': 0, 'Low': 22.5, 'High': 45}
selected_tilt = tilt_options[input_tilt]

spaces = [
    {
        "area": inputs['north_area'],           # North-facing roof section (m2)
        "tilt": selected_tilt,         # Typical roof pitch
        "azimuth": 0,         # North facing (optimal for Southern Hemisphere)
        "mounting_type": "roof_flush"
    },
    {
        "area": inputs['east_area'],           # East-facing roof section
        "tilt": selected_tilt,
        "azimuth": 90,        # East facing
        "mounting_type": "roof_flush"
    },
    {
        "area": inputs['west_area'],           # West-facing roof section
        "tilt": selected_tilt,
        "azimuth": 270,       # West facing
        "mounting_type": "roof_flush"
    },
    {
        "area": inputs['south_area'],           # South-facing roof section (will be excluded as inefficient)
        "tilt": selected_tilt,
        "azimuth": 180,       # South facing
        "mounting_type": "roof_flush"
    }
]




In [79]:
def calculate_max_panels_for_space(space, panel_type, mounting_type):
    """
    Parameters:
    - space: dict with space details including area
    - panel_type: string, type of panel
    - mounting_type: string, type of mounting (e.g., 'roof_flush', 'ground_fixed')
    
    Returns:
    - int, maximum number of panels
    """
    # Get panel dimensions
    panel_dimensions = panel_types[panel_type]["dimensions"]
    panel_area = panel_dimensions[0] * panel_dimensions[1]  # m²
    
    # Get ground coverage ratio based on mounting type
    gcr = mounting_configurations[mounting_type]["ground_coverage_ratio"]
    
    # Calculate effective area needed per panel including spacing
    effective_area_needed = panel_area / gcr
    
    # Calculate max panels for the space
    max_panels = math.floor(space["area"] / effective_area_needed)
    
    return max_panels

Calculate the maximum panels for each roof orientation

In [80]:
# Intialise outputs for each space
total_panels = {
    'north': 0,
    'south':0, 
    'east':0, 
    'west':0
}

for i, space in enumerate(spaces):
        # Check if space should be excluded based on orientation
        space_azimuth = space['azimuth']
        
        if 315 <= space_azimuth or space_azimuth < 45:
            direction = 'north'
        elif 45 <= space_azimuth < 135:
            direction = 'east' 
        elif 135 <= space_azimuth < 225:
             direction = 'south'
        else: 
             direction = 'west'

        # Calculate max panels for this space
        max_panels = calculate_max_panels_for_space(
            space=space,
            panel_type='monocrystalline',
            mounting_type=space['mounting_type']
        )
        
        # Add to total panels and capacity
        total_panels[direction] += max_panels

### Fixed Variables

For use in the linear program

In [ ]:

# Parameters (These values are examples and should be adjusted based on your system)

# Starting yeah for the data
start_year = inputs['start_year']
State = inputs['State']

# Estimated Asset lifetimes
panel_lifespan = specs['panel_lifespan']        # Estimated lifetime of solar panels
battery_lifespan = specs['battery_lifespan']    # Estimated lifetime of BESS

# Other variables
discount_rate = specs['discount_rate']/100        # Discount rate [0,1]
om_percent = specs['om_cost']/100                  # Operation and maintenance cost [0,1]  

# Variable Bounds (for linear program)
maxBatteries = inputs['max_batteries']        # Maximum number of batteries


# capital recovery factors for assets
crf_solar = (discount_rate *(1 + discount_rate)**panel_lifespan) / ((1 + discount_rate)**(panel_lifespan) - 1)
crf_bat = (discount_rate *(1 + discount_rate)**battery_lifespan) / ((1 + discount_rate)**(battery_lifespan) - 1)


# Functional Battery params
SOC_min = 0                                     # Minimum state-of-charge (%)
SOC_max = specs['battery_dod']/100              # Maximum battery capacity (%) [0,1]
B_cap = specs['battery_capacity']               # Capacity battery energy (kWh)
B_rate = specs['battery_power']                 # Battery power Rating (kW)
RTE = specs['round_trip_efficiency']/100        # Round trip efficiency (RTE) [0,1]
Ef_c =math.sqrt(RTE)                            # Charging efficiency 
Ef_d =1/math.sqrt(RTE)                          # Discharging efficiency 
cycles_80 = specs['cycles_80']                                # No. of full battery cycles until 80% battery capacity remaining
cycles = cycles_80 * battery_lifespan           # Number of cycles over shortest asset lifetime 

# Solar Panel rating (kW)
R_panel = (panel_types[specs['panel_type']]['watts_per_panel'])/1000

# Financial Params
capital_bat = specs['battery_cost'] * crf_bat               # Annualised Capital of battery install ($/kWh)
om_bat = om_percent * capital_bat                           # O&M of battery ($/kWh). Approx 1.5% of Capex
C_bat = capital_bat + om_bat                                # Install cost of battery, including O&M ($/kWh)

capital_solar = specs['solar_cost'] * crf_solar                     # Annualised Capital of solar install ($/kWp)
om_solar = om_percent * capital_solar                       # O&M of solar ($/kWh)
C_solar = capital_solar + om_solar                          # Install cost of solar panel, including O&M ($/kWh)

# Grid connection params
G_max = inputs['grid_connection']                           # Maximum allowed grid supply (kWh)

PPA = (peak_energy_price/100+additional_energy_charges/100) - 0.1     # $/kWh estimated ppa of 0.1$/kWh below average energy price

# Site Coordinates 
s = inputs['location']                                      # Extracting coordinates from dataframe
site_coords = tuple(map(float, s.split(',')))               # site coordinates as a tuple


# Panel Mounting Type
if specs['mounting_type'] == 'flush':
    pv_type = 'roof_flush'
else:
    pv_type = 'roof_tilt'

Region location for NEM data

In [82]:
nem_state = get_nem_region(site_coords[0], site_coords[1])

Priority Weightings

In [83]:
# Priority Weightings
emissions_priority  = inputs['emissions_priority']
cost_priority = inputs['cost_priority']
total = emissions_priority + cost_priority

# Normalised Weightings
w_emis = emissions_priority / total
w_cost = cost_priority / total